In [40]:
# Import data

import pandas as pd
import os
import numpy as np

In [47]:
# Specify dataset directory

data_dir: str = '../dataset/CS2_36-38'
cleaned = '../dataset/cleaned'
df = pd.DataFrame()

current = 'Current(A)'
voltage = 'Voltage(V)'
min_voltage = 3.8
max_voltage = 4.2

In [49]:
for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
        df_temp = df_temp[[current, voltage]]
        df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]

        df = pd.concat([df, df_temp])

In [50]:
print(df)
df.to_csv(f'{cleaned}/cleaned.csv')


       Current(A)  Voltage(V)
15       0.550153    3.800297
16       0.549789    3.802402
17       0.549971    3.804508
18       0.549971    3.806613
19       0.550153    3.808881
...           ...         ...
14676   -1.099536    3.869796
14677   -1.099536    3.856518
14678   -1.099536    3.845182
14679   -1.099355    3.835143
14680   -1.099355    3.826074

[391515 rows x 2 columns]


In [ ]:
# for i in range(len(df)):
